# Parameterstudy with interactive elements

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Sep  4 14:06:20 2020

@author: radmann, schwaericke, jueterbock
"""

##############################################################################
# Schalldämpfer Parameterstudie
##############################################################################

from __future__ import print_function
import numpy as np, numpy
from numpy import NaN
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

plt.rc('text', usetex=True)
from Cavity import Cavity2D, Cavity3D 
from Temperature import Temperature
from Ducts import Duct, Duct3D, DuctElement, DuctElement3D
from Fluid import Fluid 
from Material import Material
from Plate import SimplePlate, SimplePlate3D
from Linings import SinglePlateResonator, SinglePlateResonator3D

# frequency
f = np.linspace(100,2500,300)

# temperature
# Tn=widgets.FloatSlider(min=-50, max=50, step=25, value=25)
Tn=fixed(22)

# plate height
Hpn=widgets.FloatSlider(min=3e-5, max=3e-4, step=1e-5, value=1.1e-4, readout_format='.1e')

# duct height
# Hdn=widgets.FloatSlider(min=0.06, max=0.08, step=.02, value=.06)
Hdn=fixed(0.06)

# cavity height
# Heightn=widgets.FloatSlider(min=0, max=1, step=.01, value=.04)
Heightn=fixed(0.028)

# cavity/plate length
# Lengthn=widgets.FloatSlider(min=0, max=1, step=.01, value=.4)
Lengthn=widgets.Dropdown(options=[('4x4', 0.04975), ('6x4', 0.03217)])
# Depthn = widgets.Dropdown(options=[('1 chamber', 0.017),('4 chambers', 0.077)])
Depthn = fixed(0.077)
# material
E_modulen = widgets.FloatSlider(min=1e9, max=9e10, step=1e9, value=2.9e10, readout_format='.1e')
damping_factorn = widgets.FloatSlider(min=0, max=1, step=.005, value=.51)

# flow
Flown=widgets.FloatSlider(min=-.3, max=.31, step=.1, value=0.0)


def parameterstudy(length,depth,hd,hp,height,E_module,damping_factor,temperature,flow):
    ##########################################################################################################################
    # 2D Simulation ##########################################################################################################
    ##########################################################################################################################
    temp = Temperature(C=temperature)

    fluid = Fluid(temperature=temp)
     
    material = Material(rho = 1150, mu = .48, E = lambda freq, temp: E_module*(1+1j*damping_factor))
    
    plate = SimplePlate(hp=hp, material=material, temperature=temp)

    # number of modes
    R = np.ceil(2*np.max(f)*length/np.real(fluid.c))
    J = np.ceil(np.sqrt(np.max(f)*2*length**2/np.pi*
                        np.sqrt(plate.mass()/np.real(plate.bendingstiffness(np.max(f), temp)))))

    # plate modes
    j = np.arange(1, J+5, 1)
    l = np.arange(1, J+5, 1)

    # cavity modes
    r = np.arange(0, 5*R+J, 1)
    t = np.arange(0, 5, 1)
    # cavity = Cavity2D(height=height, r=r, t=t, medium=fluid, zetars=0.1)
    cavity = Cavity2D(height=height, r=r, t=t, medium=fluid)

    # lining = SinglePlateResonator(length=length, depth=1, j=j, l=l, t=t, plate=plate, cavity=cavity)
    lining = SinglePlateResonator(length=length, j=j, l=l, t=t, plate=plate, cavity=cavity)

    ductelement = DuctElement(lining=lining, medium=fluid, M=flow)

    duct = Duct(freq=f, height_d=hd, elements=[ductelement])

    N_cavities = (4 if length == .04975 else (6 if length == 0.03217 else NaN))
    tl_2D = duct.tl() #* N_cavities

    ##########################################################################################################################
    # 3D Simulation ##########################################################################################################
    ##########################################################################################################################
    plate = SimplePlate3D(hp=hp, material=material, temperature=temp)
    # depth = 0.017 # depth (i would call it width) try 17mm (one cavity) or 77mm (4 cavities)
    
    # number of modes
    J = np.ceil(np.sqrt(np.max(f)*2*length**2/np.pi*np.sqrt(plate.mass()/np.real(plate.bendingstiffness(np.max(f), temp)))))
    K = np.ceil(np.sqrt(np.max(f)*2*depth**2/np.pi*np.sqrt(plate.mass()/np.real(plate.bendingstiffness(np.max(f), temp)))))
    R = np.ceil(2*np.max(f)*length/np.real(fluid.c))
    S = np.ceil(2*np.max(f)*depth/np.real(fluid.c))
    T = np.ceil(2*np.max(f)*height/np.real(fluid.c))
    
    # plate modes
    j = np.arange(1, J+1, 1, dtype='int')
    k = np.arange(1, K+1, 1, dtype='int')
    l = np.arange(1, J+1, 1, dtype='int')
    n = np.arange(1, K+1, 1, dtype='int')
    
    # cavity modes
    r = np.arange(0, R+J, 1, dtype='int')
    s = np.arange(0, S+K, 1, dtype='int')
    t = np.arange(0, 5*T, 1, dtype='int')
    
    cavity = Cavity3D(height=height, r=r, s=s, t=t, medium=fluid)
    lining = SinglePlateResonator3D(length=length, depth=depth, j=j, k=k, l=l, n=n, t=t, plate=plate, cavity=cavity)
    ductelement = DuctElement3D(lining=lining, medium=fluid, M=flow)
    duct = Duct3D(freq=f, height_d=hd, elements=[ductelement])
    tl_3D = duct.tl() #* N_cavities # transmission loss for 1 cavity multiplied by number of cavities
    
    plt.figure(figsize=(12,6))
    # plt.xscale('log')       
    plt.plot(f,tl_2D,label='2D')
    plt.plot(f,tl_3D,label='3D')
    plt.title("")
    plt.xlabel('$f$ [Hz]')
    plt.ylabel('TL [dB]')
    plt.ylim(0,15)
    
    plt.legend()
    return

interact(parameterstudy, length=Lengthn, depth=Depthn, hd=Hdn, hp=Hpn, height=Heightn, E_module=E_modulen, damping_factor=damping_factorn, temperature=Tn, flow=Flown)

plt.show()


interactive(children=(Dropdown(description='length', options=(('4x4', 0.04975), ('6x4', 0.03217)), value=0.049…